In [1]:
%load_ext dotenv
%dotenv

In [2]:
import pandas as pd
from ch07_tools import get_answer
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_correctness
from tqdm import tqdm
from utils import neo4j_driver


/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
get_answer("How many movies are there?")

'There are 38 movies in total.'

In [4]:
test_data = pd.read_csv("../data/benchmark_data.csv", delimiter=";")
test_data.head()

,question,cypher
0,How many movies are there?,MATCH (m:Movie) RETURN count(*) AS movies
1,Who acted in the most movies?,MATCH (p:Person)-[:ACTED_IN]->(m:Movie)\nRETUR...
2,Which movies released between 1990 and 2000 ha...,MATCH (m:Movie) WHERE m.released >= 1990 AND m...
3,Who has the longest name among all actors?,MATCH (p:Person)-[:ACTED_IN]->(:Movie) RETURN ...
4,List people born before 1940.,MATCH (p:Person) WHERE p.born < 1940 RETURN p....


In [5]:
answers = []
ground_truths = []

for i, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing rows"):
    ground_truth, _, _ = neo4j_driver.execute_query(row["cypher"])
    ground_truths.append([el.data() for el in ground_truth])
    try:
        answer = get_answer(row["question"])
    except Exception:
        answer = None
    answers.append(answer)

Processing rows:  80%|████████  | 8/10 [02:27<00:28, 14.45s/it]Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n    MATCH (a:Person)-[:ACTED_IN]->(m:Movie)\n    OPTIONAL MATCH (m)<-[:ACTED_IN]-(a:Person)\n    OPTIONAL MATCH (m)<-[:DIRECTED]-(d:Person)\n    WHERE toLower(a.name) CONTAINS $actor\n    RETURN m AS movie, collect(a.name) AS cast, collect(d.name) AS directors\n    '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n    MATCH (a:Person)-[:ACTED_IN]->(m:Movie)\n    

In [6]:
test_data['ground_truth'] = str(ground_truths)
test_data['answer'] = answers

In [7]:
test_data

,question,cypher,ground_truth,answer
0,How many movies are there?,MATCH (m:Movie) RETURN count(*) AS movies,"[[{'movies': 38}], [{'p.name': 'Tom Hanks', 'm...",There are 38 movies in total.
1,Who acted in the most movies?,MATCH (p:Person)-[:ACTED_IN]->(m:Movie)\nRETUR...,"[[{'movies': 38}], [{'p.name': 'Tom Hanks', 'm...",The actor who acted in the most movies is Tom ...
2,Which movies released between 1990 and 2000 ha...,MATCH (m:Movie) WHERE m.released >= 1990 AND m...,"[[{'movies': 38}], [{'p.name': 'Tom Hanks', 'm...",The only movie released between 1990 and 2000 ...
3,Who has the longest name among all actors?,MATCH (p:Person)-[:ACTED_IN]->(:Movie) RETURN ...,"[[{'movies': 38}], [{'p.name': 'Tom Hanks', 'm...",None
4,List people born before 1940.,MATCH (p:Person) WHERE p.born < 1940 RETURN p....,"[[{'movies': 38}], [{'p.name': 'Tom Hanks', 'm...",The people born before 1940 are:\n\n- Max von ...
5,Who was born in 1965 and has directed a movie?,MATCH (p:Person)-[:DIRECTED]->(m:Movie) WHERE ...,"[[{'movies': 38}], [{'p.name': 'Tom Hanks', 'm...",The people born in 1965 who have directed a mo...
6,Who acted in Top Gun?,MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title:...,"[[{'movies': 38}], [{'p.name': 'Tom Hanks', 'm...","The actors who acted in ""Top Gun"" are Val Kilm..."
7,"List the movies with a title starting with ""The""","MATCH (m:Movie) WHERE m.title STARTS WITH ""The...","[[{'movies': 38}], [{'p.name': 'Tom Hanks', 'm...","The movies with a title starting with ""The"" ar..."
8,How many movies did Tom Hanks appear in?,"MATCH (p:Person {name: ""Tom Hanks""})-[:ACTED_I...","[[{'movies': 38}], [{'p.name': 'Tom Hanks', 'm...",Tom Hanks appeared in 12 movies.
9,Who played in the Matrix?,MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title:...,"[[{'movies': 38}], [{'p.name': 'Tom Hanks', 'm...","The main cast members of ""The Matrix"" include ..."


In [8]:
dataset = Dataset.from_pandas(test_data.fillna("I don't know"))
result = evaluate(
    dataset,
    metrics=[
        answer_correctness
    ]
)

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
result

{'answer_correctness': 0.6850}